# 인식모델 학습

- 학습데이터 : roboflow
- 학습환경 : Colab(GPU)

In [1]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |████████████████▎               | 834.1 MB 1.2 MB/s eta 0:10:44tcmalloc: large alloc 1147494400 bytes == 0x665a6000 @  0x7faa18ad7615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████▋           | 1055.7 MB 1.2 MB/s eta 0:07:46tcmalloc: large alloc 1434370048 bytes == 0x3dce000 @  0x7faa18ad7615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |██████████████████████████▏     | 1

In [2]:
import torch

In [3]:
torch.__version__


'1.11.0+cu113'

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
!git clone https://github.com/ultralytics/yolov5/

Cloning into 'yolov5'...
remote: Enumerating objects: 12633, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 12633 (delta 105), reused 111 (delta 66), pack-reused 12467
Receiving objects: 100% (12633/12633), 12.83 MiB | 33.87 MiB/s, done.
Resolving deltas: 100% (8704/8704), done.


In [6]:
!pip install -qr /content/yolov5/requirements.txt

     |████████████████████████████████| 1.6 MB 39.6 MB/s 


In [7]:
# !curl -L "https://app.roboflow.com/ds/2hSushzU6S?key=abpmPbxuMS" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip              #신호등 횡단보도
# !curl -L "https://app.roboflow.com/ds/wwT5ZDjveE?key=1PQh3cy7wv" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip              #v3
# !curl -L "https://app.roboflow.com/ds/bnJ7I9arxW?key=dyNAegSG7L" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip              # 보행자
# !curl -L "https://app.roboflow.com/ds/7w42P6yFLI?key=UMq1FxjPWU" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip #내 자료
# !curl -L "https://app.roboflow.com/ds/r7UBar6c4t?key=5QaUdhGBLa" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip # 병합자료
# !curl -L "https://app.roboflow.com/ds/r7UBar6c4t?key=5QaUdhGBLa" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!curl -L "https://app.roboflow.com/ds/bnJ7I9arxW?key=dyNAegSG7L" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip #보행자

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/labels/84e0c6e5-69c178b9_jpg.rf.221e77bd1a8f56fd73a22bc8c4bf40de.txt  
 extracting: train/labels/84e0c6e5-69c178b9_jpg.rf.80a0852cfc876bd6715c27980aa3003d.txt  
 extracting: train/labels/84e0c6e5-69c178b9_jpg.rf.cce819411b5a67414992c5c8cd299897.txt  
 extracting: train/labels/84e0c6e5-b4b14a9b_jpg.rf.22bf4602e5cfd63a3fcb2117071728c6.txt  
 extracting: train/labels/84e0c6e5-b4b14a9b_jpg.rf.98ee641c5ed9a00a28c3a0d5c20dafbe.txt  
 extracting: train/labels/84e0c6e5-b4b14a9b_jpg.rf.a4cad83494870188e6329457d40cbd04.txt  
 extracting: train/labels/84ee29aa-2c01b4b4_jpg.rf.1f298a9b72a941ed610c54e933d365c7.txt  
 extracting: train/labels/84ee29aa-2c01b4b4_jpg.rf.550c7ffd128d1fb9c1f35cf00efd3405.txt  
 extracting: train/labels/84ee29aa-2c01b4b4_jpg.rf.8e31981a9a8a376d69d3a6cf378e0d61.txt  
 extracting: train/labels/84f3d83d-59928c47_jpg.rf.2dad6810a174b6574a0a5d7430ae7242.txt  
 extracting: train/labels/84f3d83d-59928c47_jpg.rf.6fcb28ec61a03

In [8]:
!python /content/yolov5/train.py \
--img 416 \
--batch 32 \
--epochs 30 \
--data /content/data.yaml \
--weights yolov5s.pt \
--name res-pedes \
--cfg /content/yolov5/models/yolov5s.yaml

train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=res-pedes, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-155-g489920a Python-3.7.14 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a

In [ ]:
!python /content/yolov5/export.py \
--weights /content/yolov5/runs/train/res-cross-pedes2/weights/best.pt \
--include onnx \
--simplify

export: data=yolov5/data/coco128.yaml, weights=['/content/yolov5/runs/train/res-cross-pedes2/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v6.2-149-g77dcf55 Python-3.7.14 torch-1.11.0+cu113 CPU

Fusing layers... 
YOLOv5l summary: 267 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs

PyTorch: starting from /content/yolov5/runs/train/res-cross-pedes2/weights/best.pt with output shape (1, 25200, 8) (88.5 MB)
requirements: YOLOv5 requirement "onnx" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per ['onnx']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: st

In [ ]:
# !python /content/yolov3/train.py --weights yolov3.pt --img 640 --conf 0.25 --source /content/train

In [ ]:
# 테스트 데이터를 이용한 객체 탐지
# --conf 0.1 \ 바운딩박스로 표기되는 객체 검출 쿼터를 배제 (통상 0.5 이상)

!python /content/yolov5/detect.py \
--weights /content/yolov5/runs/train/res-crosswalk/weights/best.pt \
--img 416 \
--source /content/test/images

In [ ]:
# 영상 데이터를 이용한 객체 탐지
!python /content/yolov5/detect.py \
--weights /content/yolov5/runs/train/res-cross-pedes2/weights/best.pt \
--source /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/crosswalk.mp4 \
--conf 0.5
# --save-txt

detect: weights=['/content/yolov5/runs/train/res-cross-pedes2/weights/best.pt'], source=/content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/crosswalk.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-149-g77dcf55 Python-3.7.14 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5l summary: 267 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
video 1/1 (1/1049) /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/crosswalk.mp4: 384x640 (no detections), 38.8ms
video 1/1 (2/1049) /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/crosswalk.m

In [ ]:
# 모델 백업
!cp -r /content/yolov5/runs/train/res-cross-pedes2 /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/모델백업

In [ ]:
!python /content/yolov5/detect.py \
--weights /content/yolov5/runs/train/res-crosswalk/weights/best.pt \
--img 416 \
--source /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/교차로사진

In [ ]:
# 웹캠 데이터를 이용한 실시간 객체 탐지
!python /content/yolov5/detect.py \
--weights /content/yolov5/runs/train/res-crosswalk/weights/best.pt \
--source  \
--conf 0.5

detect: weights=['/content/yolov5/runs/train/res-crosswalk/weights/best.pt'], source=0, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-143-g63368e7 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show() image displays
cv2.imshow() is disabled in Docker environments
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 255, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 250, in main
    run(**vars(opt))
  F

In [ ]:
# 영상 데이터를 이용한 객체 탐지( 복수 모델 )
!python /content/yolov5/detect.py \
--weights /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/모델백업/res-crosswalk/weights/best.pt /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/보행자모델백업/weights/best.pt  \
--source /content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/crosswalk.mp4 \
--conf 0.3

detect: weights=['/content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/모델백업/res-crosswalk/weights/best.pt', '/content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/보행자모델백업/weights/best.pt'], source=/content/drive/MyDrive/딥러닝_학습/미니프로젝트/차선인식모델/crosswalk.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-137-gfda8aa5 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs
Fusing layers... 
YOLOv5x summary: 322 layers, 86186872 parameters, 0 gradie

In [ ]:
!python /content/yolov5/export.py --weights /content/yolov5/runs/train/res-crosswalk/weights/best.pt --include torchscript

export: data=yolov5/data/coco128.yaml, weights=['/content/yolov5/runs/train/res-crosswalk/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript']
YOLOv5 🚀 v6.2-143-g63368e7 Python-3.7.14 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs

PyTorch: starting from /content/yolov5/runs/train/res-crosswalk/weights/best.pt with output shape (1, 25200, 7) (165.0 MB)

TorchScript: starting export with torch 1.12.1+cu113...
TorchScript: export success ✅ 19.8s, saved as /content/yolov5/runs/train/res-crosswalk/weights/best.torchscript (329.5 MB)

Export complete (29.8s)
Results saved to /content/yolov5/runs/train/res-crosswalk/weights
Detect:          python detect.py 